In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/mlds

Mounted at /content/drive
/content/drive/MyDrive/mlds


In [2]:
import data_bitcoin as btc
import tasker_link_prediction as t_lp
from splitter import splitter
from models import Baseline_node2vec
from trainer import Trainer
import yaml
import os

In [3]:
prep = False

with open('config/config_BTC_OTC_baseline.yaml', 'r') as file:
    config = yaml.safe_load(file)

model_path = config['model_path']

if not os.path.exists(model_path):
    os.mkdir(model_path)
    os.mkdir(model_path + "best_checkpoints/")
    os.mkdir(model_path + "latest_checkpoints/")

with open(model_path + 'config.yaml', 'w') as file:
    yaml.dump(config, file)

config

{'adam_config': {'adam_beta_1': 0.9,
  'adam_beta_2': 0.999,
  'adam_epsilon': 1e-07,
  'adam_learning_rate': 0.005},
 'model_path': 'models/BTC_OTC_N2V_baseline/',
 'classifier_hidden_size': 16,
 'ffn_fusion_size': 8,
 'ffn_hidden_size': 8,
 'gcn_fusion_size': 16,
 'spatial_hidden_size': 16,
 'spatial_input_dim': 64,
 'temporal_hidden_size': 16,
 'temporal_input_dim': 80,
 'num_epochs': 1000,
 'patience': 10,
 'major_threshold': None,
 'train_proportion': 0.7,
 'dev_proportion': 0.1,
 'data_path': 'data/BTC-OTC/',
 'prep_data_path': 'prep_data/BTC_OTC_neg/'}

In [4]:
data = btc.Bitcoin_Dataset(config['data_path'])
tasker = t_lp.Link_Pred_Tasker(data, path=config['prep_data_path'], prep=prep,
                               embs_dim=config['spatial_input_dim'], temp_dim=int(config['temporal_input_dim']/10),
                               major_threshold=config['major_threshold'], smart_neg_sampling=True)

splitter_as = splitter(tasker, train_proportion = config['train_proportion'], dev_proportion = config['dev_proportion'])

model= Baseline_node2vec(config['spatial_input_dim'])

trainer = Trainer(model=model, splitter=splitter_as, model_path=model_path, adam_config=config['adam_config'], patience=config['patience'])
print(trainer.patience)

Dataset splits sizes:  train 86 dev 14 test 28
10


In [5]:
trainer.train(config['num_epochs'])

Epoch 1, Step 1, Loss: 0.08455858379602432
Epoch 1, Step 2, Loss: 0.07383275777101517
Epoch 1, Step 3, Loss: 0.06093551218509674
Epoch 1, Step 4, Loss: 0.060105100274086
Epoch 1, Step 5, Loss: 0.04772182181477547
Epoch 1, Step 6, Loss: 0.04466985911130905
Epoch 1, Step 7, Loss: 0.03135096654295921
Epoch 1, Step 8, Loss: 0.044529493898153305
Epoch 1, Step 9, Loss: 0.04826757311820984
Epoch 1, Step 10, Loss: 0.03773126378655434
Epoch 1, Step 11, Loss: 0.03320885822176933
Epoch 1, Step 12, Loss: 0.03334999084472656
Epoch 1, Step 13, Loss: 0.03090774081647396
Epoch 1, Step 14, Loss: 0.029473483562469482
Epoch 1, Step 15, Loss: 0.03207558020949364
Epoch 1, Step 16, Loss: 0.03376711532473564
Epoch 1, Step 17, Loss: 0.036245182156562805
Epoch 1, Step 18, Loss: 0.03611339256167412
Epoch 1, Step 19, Loss: 0.0337519533932209
Epoch 1, Step 20, Loss: 0.03942738100886345
Epoch 1, Step 21, Loss: 0.02975546009838581
Epoch 1, Step 22, Loss: 0.02868075482547283
Epoch 1, Step 23, Loss: 0.036116160452365

In [6]:
trainer.restore_best_checkpoint()
trainer.validate('Validation')
trainer.validate('Test')

Validation Metric MRRs: 0.047429348704693584
Validation Metric MAPs: 0.007557749404694969
Test Metric MRRs: 0.054012917455391794
Test Metric MAPs: 0.014058892024495624


(0.054012917455391794, 0.014058892024495624)